### 0.Import package

In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import requests, json
from pandas import json_normalize
import json


### 1. Get Data

In [2]:
from fdrsapi import api_function,baseline

In [3]:
f = open("api_key.txt", "r")
api_key=f.readline()
years=["2016","2017","2018","2019","2020"]
kpi_code=["KPI_DonBlood_Tot","KPI_TrainFA_Tot","KPI_ReachDRER_D_Tot","KPI_ReachDRER_I","KPI_ReachDRR_D_Tot","KPI_ReachDRR_I","KPI_ReachLTSPD_D_Tot","KPI_ReachLTSPD_I","KPI_ReachS_D_Tot","KPI_ReachS_I","KPI_ReachL_D_Tot","KPI_ReachL_I","KPI_ReachM_D_Tot","KPI_ReachM_I","KPI_ReachCTP_D_Tot","KPI_ReachCTP_I","KPI_ReachSI_D_Tot","KPI_ReachSI_I","KPI_ReachH_D_Tot","KPI_ReachH_I","KPI_ReachWASH_I","KPI_ReachWASH_D_Tot","KPI_ReachDRER_CPD","KPI_ReachLTSPD_CPD","KPI_ReachDRR_CPD","KPI_ReachS_CPD","KPI_ReachL_CPD", "KPI_ReachH_CPD","KPI_ReachWASH_CPD","KPI_ReachM_CPD","KPI_ReachCTP_CPD","KPI_ReachSI_CPD","KPI_GB_Tot","KPI_pr_sex","KPI_sg_sex","KPI_PeopleVol_Tot","KPI_PStaff_Tot","KPI_noLocalUnits","KPI_IncomeLC","KPI_IncomeLC_CHF","KPI_expenditureLC","KPI_expenditureLC_CHF"]
kpi_float=kpi_code.copy()
kpi_float.remove("KPI_pr_sex")
kpi_float.remove("KPI_sg_sex")
time_series=api_function(years,kpi_code,kpi_float,api_key)
time_series

2016
2017
2018
2019
2020
number of KPI downloaded: 42


,KPI ID,KPI_DON_code,NSO_DON_name,NSO_ZON_name,iso_3,KPI_Year,KPI_DonBlood_Tot,KPI_GB_Tot,KPI_IncomeLC,KPI_IncomeLC_CHF,...,KPI_ReachS_I,KPI_ReachWASH_CPD,KPI_ReachWASH_D_Tot,KPI_ReachWASH_I,KPI_TrainFA_Tot,KPI_expenditureLC,KPI_expenditureLC_CHF,KPI_noLocalUnits,KPI_pr_sex,KPI_sg_sex
0,DAF0012016,DAF001,Afghan Red Crescent,Asia Pacific,AFG,2016,178.0,NaN,1.293236e+09,1.994429e+07,...,NaN,0.0,0.0,NaN,4020.0,1.332000e+09,2.054210e+07,58.0,NaN,NaN
1,DAL0012016,DAL001,Albanian Red Cross,Europe and Central Asia,ALB,2016,6181.0,NaN,1.681344e+08,1.348774e+06,...,NaN,0.0,0.0,NaN,53259.0,1.355361e+08,1.087271e+06,39.0,NaN,NaN
2,DDZ0012016,DDZ001,Algerian Red Crescent,Middle East and North Africa,DZA,2016,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,NaN,NaN
3,DUS0012016,DUS001,American Red Cross,Americas,USA,2016,2720640.0,NaN,2.660138e+09,2.604914e+09,...,0.0,0.0,0.0,NaN,2302312.0,2.721539e+09,2.665040e+09,264.0,NaN,NaN
4,DAD0012016,DAD001,Andorran Red Cross,Europe and Central Asia,AND,2016,1028.0,NaN,1.892810e+06,2.029149e+06,...,NaN,101.0,101.0,0.0,2257.0,8.871990e+05,9.511040e+05,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,DVE0012020,DVE001,Venezuelan Red Cross,Americas,VEN,2020,428.0,11.0,6.635811e+10,0.000000e+00,...,NaN,302188.0,302188.0,NaN,2984.0,5.275225e+10,0.000000e+00,42.0,Male,Male
956,DVN0012020,DVN001,Vietnam Red Cross Society,Asia Pacific,VNM,2020,1405336.0,5.0,1.479656e+11,5.918624e+06,...,NaN,127406.0,127406.0,NaN,62961.0,7.441142e+10,2.976457e+06,17127.0,Female,Male
957,DYE0012020,DYE001,Yemen Red Crescent Society,Middle East and North Africa,YEM,2020,NaN,6.0,2.823305e+09,4.452352e+06,...,NaN,415482.0,415482.0,NaN,2495.0,2.352443e+09,3.709803e+06,31.0,Male,Male
958,DZM0012020,DZM001,Zambia Red Cross Society,Africa,ZMB,2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2. Run imputing missing values

In [4]:
from imp_function import imputing_mean,imputing_na

In [5]:
# drop from 0 to 2011
data_clean=time_series.query('KPI_Year >= 2012 & KPI_Year < 2021')
# filter out columns that can be ignored: 64 columns dropped (see columns 1134 to 1070)
# The only reporting year FDRS didn't count with 100% reporting was 2019. Therefore, lets divide the dataset in 2: 
# 2012-2017, 2018-2020. This facilitates the code as KPI_WasSubmitted variable (which is being used to filter the data) 
# was implemented in 2017 and all years before have NaN 
# 1st subset:
data_clean1=data_clean.query('KPI_Year >= 2012 & KPI_Year < 2018').copy()
# 2nd subset:
data_clean2=data_clean.query('KPI_Year >= 2018').copy()
data_clean2_na_columns=data_clean2.copy()
#Imputing:People reached sections 
selected_cols=["KPI_DonBlood_Tot","KPI_TrainFA_Tot","KPI_ReachDRER_D_Tot","KPI_ReachDRER_I","KPI_ReachDRR_D_Tot","KPI_ReachDRR_I","KPI_ReachLTSPD_D_Tot","KPI_ReachLTSPD_I","KPI_ReachS_D_Tot","KPI_ReachS_I","KPI_ReachL_D_Tot","KPI_ReachL_I","KPI_ReachM_D_Tot","KPI_ReachM_I","KPI_ReachCTP_D_Tot","KPI_ReachCTP_I","KPI_ReachSI_D_Tot","KPI_ReachSI_I","KPI_ReachH_D_Tot","KPI_ReachH_I","KPI_ReachWASH_I","KPI_ReachWASH_D_Tot","KPI_ReachDRER_CPD","KPI_ReachLTSPD_CPD","KPI_ReachDRR_CPD","KPI_ReachS_CPD","KPI_ReachL_CPD", "KPI_ReachH_CPD","KPI_ReachWASH_CPD","KPI_ReachM_CPD","KPI_ReachCTP_CPD","KPI_ReachSI_CPD"]
imputing_mean(data_clean2_na_columns,selected_cols,data_clean2)
# Imputing : NS Finance and Parnership & NS Governance and Structure Section
selected_cols_2d=["KPI_GB_Tot","KPI_pr_sex","KPI_sg_sex","KPI_PeopleVol_Tot","KPI_PStaff_Tot","KPI_noLocalUnits","KPI_IncomeLC","KPI_IncomeLC_CHF","KPI_expenditureLC" ,"KPI_expenditureLC_CHF"]
# 18.11.21Comment : After reviewing with the Team, "supported1" and "received_support1" have been imputed seperatly in the script FDRS Network Data Transformation
imputing_na(data_clean2_na_columns,selected_cols_2d)
# concatenate the two sub_datasets
fdrs_na_columns = [data_clean1, data_clean2_na_columns]
fdrs_data_fdrs_na_columns = pd.concat(fdrs_na_columns)


KPI_DonBlood_Tot
KPI_TrainFA_Tot
KPI_ReachDRER_D_Tot
KPI_ReachDRER_I
KPI_ReachDRR_D_Tot
KPI_ReachDRR_I
KPI_ReachLTSPD_D_Tot
KPI_ReachLTSPD_I
KPI_ReachS_D_Tot
KPI_ReachS_I
KPI_ReachL_D_Tot
KPI_ReachL_I
KPI_ReachM_D_Tot
KPI_ReachM_I
KPI_ReachCTP_D_Tot
KPI_ReachCTP_I
KPI_ReachSI_D_Tot
KPI_ReachSI_I
KPI_ReachH_D_Tot
KPI_ReachH_I
KPI_ReachWASH_I
KPI_ReachWASH_D_Tot
KPI_ReachDRER_CPD
KPI_ReachLTSPD_CPD
KPI_ReachDRR_CPD
KPI_ReachS_CPD
KPI_ReachL_CPD
KPI_ReachH_CPD
KPI_ReachWASH_CPD
KPI_ReachM_CPD
KPI_ReachCTP_CPD
KPI_ReachSI_CPD
KPI_GB_Tot
KPI_pr_sex
KPI_sg_sex
KPI_PeopleVol_Tot
KPI_PStaff_Tot
KPI_noLocalUnits
KPI_IncomeLC
KPI_IncomeLC_CHF
KPI_expenditureLC
KPI_expenditureLC_CHF


### 3. Post method - interact with Imp_Var

Total figures will change in the main page:

https://data-staging.ifrc.org/fdrs


#### 1. Testing

Testing with imputing Indonesia Total Vol in 2020   
Total Vol in 2020 : 10.1 before imputing  

In [ ]:
url = 'https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=21e401ae-6b35-404b-a72a-b74cce66dee3&kpicode=KPI_PeopleVol_Tot_IP&year=2020&don_code=DID001&value=384647&user=simon.weiss@ifrc.org'

In [ ]:
r = requests.post(url)

In [ ]:
print(r.text)

"Ok"


Working: 
Total Vol 10.5 after imputing   
Value added https://data-api-staging.ifrc.org/api/KpiImputedValue?kpicode=KPI_PeopleVol_Tot_IP&year=2020&apiKey=21e401ae-6b35-404b-a72a-b74cce66dee3

#### Function

Years to impute:   
(keeping 2016 for frontend)

In [6]:
years=[2016,2017,2018,2019,2020]

Variables to impute :   
total in front end

In [11]:
kpi_code=["KPI_PeopleVol_Tot","KPI_PStaff_Tot","KPI_DonBlood_Tot","KPI_TrainFA_Tot","KPI_noLocalUnits","KPI_ReachDRER_CPD","KPI_ReachLTSPD_CPD","KPI_ReachDRR_CPD","KPI_ReachS_CPD","KPI_ReachL_CPD","KPI_ReachH_CPD","KPI_ReachWASH_CPD","KPI_ReachM_CPD","KPI_ReachCTP_CPD","KPI_ReachSI_CPD","KPI_IncomeLC_CHF","KPI_expenditureLC_CHF"]

In [12]:
len(kpi_code)

17

In [ ]:
0.38

In [13]:
#fdrs_data_fdrs_na_columns_test.fillna(0,inplace=True)
for kpi in list(kpi_code):
    for year in years:
        for kpi_don_code in list(set(fdrs_data_fdrs_na_columns["KPI_DON_code"])): 
            for label, content in (fdrs_data_fdrs_na_columns[kpi][(fdrs_data_fdrs_na_columns["KPI_Year"]==year)&(fdrs_data_fdrs_na_columns["KPI_DON_code"]==kpi_don_code)]).items():
                value = content
                if type(value)==float:
                    if not np.isnan(value):
                        value=np.int64(value)
                kpi_post=kpi+"_IP"
                url = (f"https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=21e401ae-6b35-404b-a72a-b74cce66dee3&kpicode={kpi_post}&year={year}&don_code={kpi_don_code}&value={value}&user=simon.weiss@ifrc.org")
                r = requests.post(url)
                print(url,r.text)

https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=21e401ae-6b35-404b-a72a-b74cce66dee3&kpicode=KPI_PeopleVol_Tot_IP&year=2016&don_code=DCK001&value=390&user=simon.weiss@ifrc.org "Ok"
https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=21e401ae-6b35-404b-a72a-b74cce66dee3&kpicode=KPI_PeopleVol_Tot_IP&year=2016&don_code=DML001&value=6329&user=simon.weiss@ifrc.org "Ok"
https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=21e401ae-6b35-404b-a72a-b74cce66dee3&kpicode=KPI_PeopleVol_Tot_IP&year=2016&don_code=DNE001&value=nan&user=simon.weiss@ifrc.org "Ok"
https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=21e401ae-6b35-404b-a72a-b74cce66dee3&kpicode=KPI_PeopleVol_Tot_IP&year=2016&don_code=DTV001&value=nan&user=simon.weiss@ifrc.org "Ok"
https://data-api-staging.ifrc.org/api/ImputedKPI?apiKey=21e401ae-6b35-404b-a72a-b74cce66dee3&kpicode=KPI_PeopleVol_Tot_IP&year=2016&don_code=DKI001&value=nan&user=simon.weiss@ifrc.org "Ok"
https://data-api-staging.ifrc.org/api/ImputedKPI?apiKe

Time to execute all : 111min

#### To develop :

Check diff between api get KPI_Imp and fdrs_data_null in order to post only value that are new and gain time.